## Load Dataset

In [1]:
# Here we are importing all the libraries we will need to access functions
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Let's load this dataset from uiuc-cse
dataset = pd.read_csv("https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv")

# Importing and Cleaning Up the Dataset
dataset.loc[dataset["species"]=="setosa","species"]     = 0
dataset.loc[dataset["species"]=="versicolor","species"] = 1
dataset.loc[dataset["species"]=="virginica","species"]  = 2

dataset

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


## Create database for tracking mlflow models

In [2]:
import mysql.connector
from mysql.connector import Error

try:
    mydb = mysql.connector.connect(host='localhost', user='root', password='admin123')
    if mydb.is_connected():
        cursor = mydb.cursor()
        cursor.execute('DROP DATABASE IF EXISTS mlflowDB')
        cursor.execute("CREATE DATABASE mlflowDB")
        print("mlflowDB database is created")
except Error as e:
    print(e)

mlflowDB database is created


## Model training & tracking process 

In [3]:
import mlflow
import train

# Initial training variable settings
model   = train.trainANN(
        data=dataset,
        experiment_name="Question 1.a", 
        tags={
                "engineering": "Skill test ML Engineer e-fishery",
                "release.candidate": "RC1",
                "release.version": "1.0.0"
        }
)
# Run the training process
model.run(
        n_iter=5,
        run_name="run@1",
        fitted_param={
                "batch_size":20,
                "epochs":100,
                "verbose":0}
)
# Get runid for best model based on MCC
bestModel = model.getBestModel(metrics='MCC')

2022/09/15 10:06:04 INFO mlflow.tracking.fluent: Experiment with name 'Question 1.a' does not exist. Creating a new experiment.


INFO:tensorflow:Assets written to: C:\Users\LENOVO\AppData\Local\Temp\tmpmw1swqrd\model\data\model\assets
1/1 [==============================] - 0s 126ms/step
INFO:tensorflow:Assets written to: C:\Users\LENOVO\AppData\Local\Temp\tmp06zgfux5\model\data\model\assets
1/1 [==============================] - 0s 80ms/step
INFO:tensorflow:Assets written to: C:\Users\LENOVO\AppData\Local\Temp\tmpth62bcp5\model\data\model\assets
1/1 [==============================] - 0s 116ms/step
INFO:tensorflow:Assets written to: C:\Users\LENOVO\AppData\Local\Temp\tmpq_0bia2h\model\data\model\assets
1/1 [==============================] - 0s 81ms/step
INFO:tensorflow:Assets written to: C:\Users\LENOVO\AppData\Local\Temp\tmpt8pgvw_l\model\data\model\assets
1/1 [==============================] - 0s 75ms/step

Training has been completed, Log File Location: file:///./Training/SkillTaskML/mlruns/1

The best model id: a4bc07f72bb349cc93cfda5179ffc193


<Figure size 640x480 with 0 Axes>

# Registering a model

In [4]:
# Adding an MLflow Model to the Model Registry
mlflow.register_model(model_uri=f"runs:/{bestModel}/keras-ann-model", name="keras-ann-reg")

Successfully registered model 'keras-ann-reg'.
2022/09/15 10:07:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: keras-ann-reg, version 1
Created version '1' of model 'keras-ann-reg'.


<ModelVersion: creation_timestamp=1663211261178, current_stage='None', description='', last_updated_timestamp=1663211261178, name='keras-ann-reg', run_id='a4bc07f72bb349cc93cfda5179ffc193', run_link='', source='file:///./Training/SkillTaskML/mlruns/1/a4bc07f72bb349cc93cfda5179ffc193/artifacts/keras-ann-model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [5]:
from mlflow.tracking import MlflowClient

# Set models version 1 from None into Production
client = MlflowClient()
client.transition_model_version_stage(name="keras-ann-reg", version=1,stage="Production")

<ModelVersion: creation_timestamp=1663211261178, current_stage='Production', description='', last_updated_timestamp=1663211263833, name='keras-ann-reg', run_id='a4bc07f72bb349cc93cfda5179ffc193', run_link='', source='file:///./Training/SkillTaskML/mlruns/1/a4bc07f72bb349cc93cfda5179ffc193/artifacts/keras-ann-model', status='READY', status_message='', tags={}, user_id='', version='1'>